# Getting data from online APIs

## Twitter

It's relatively easy to get recent data from twitter. But you might have to pay for historical data.

The first step is go to http://apps.twitter.com to create an app. 
You'll get a consumer_key, consumer_secrete, access_token, and accces_token_secret.

These two random web pages are somewhat helpful:

https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/

https://www.geeksforgeeks.org/extraction-of-tweets-using-tweepy/

Then you connect to the twitter api as below. 

You can use my keys and tokens today. But you should create your own if you're planning to do more.

These command you execute just once:

In [ ]:
import tweepy
consumer_key = "G37xcCsQa5vkVhT9d6aFGgGWd"
consumer_secret = "KNkBbqldSqjslanKD0pThNvmhbNptlPhj9YqPVjAG3Whyvfugf"
access_token = "26928590-nWahsl79rYOGlzRWGGtidq1formdn6sy87Ea32E7U"
access_token_secret = "toZsqPKDMp1zgwdgjWVj4YPqgLzM9PcjfsXqypeaJ8Az2"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

This is how you do a research:

(Note, if you don't use `tweet_mode='extended'` then the text of long tweets will be truncated.)

In [ ]:
language = "en"
query = "#mtbos"
results = api.search(q=query, lang=language, count=100, tweet_mode='extended')

This will return a list of a results.

To extract the information you need, you'll want the ._json property. This will give you a dict.

In [ ]:
post = results[0]._json

In [ ]:
post.keys()

In [ ]:
post["full_text"]

## Propublica

Propublica makes some substantial data freely available from an online API.

You have to request a key. But it's quick and easy.

https://www.propublica.org/datastore/apis

In [ ]:
import requests
my_propub_key = "ktlRtmbsOL7c27WRGsSeUSYFJRdU04EZQZIKGSuv"
headers = {'X-API-Key': my_propub_key}

In [ ]:
import math
def count_documents_pages(propublica_query): # This function number of pages/requests to make for JSON files based on the number of query results.
    offset = 0
    initial_request = requests.get(propublica_query.format(offset), headers = headers).json()
    num_pages = math.ceil(initial_request['num_results']/20) # Each response shows 20 articles. ProPublica API supports pagination, so we need to calculate how many pages we need to request.
    print("Number of documents is {}.".format(initial_request['num_results']))
    print("Number of pages is {}.".format(num_pages))
    return num_pages

In [ ]:
propublica_query = "https://api.propublica.org/congress/v1/statements/search.json?query=%22common%20core%22&offset={0}"

In [ ]:
count_documents_pages(propublica_query)

In [ ]:
offset = 0
initial_request = requests.get(propublica_query.format(offset), headers = headers).json()

In [ ]:
initial_request.keys()

This tells us how many matches our search found.
But it only gives us the first 20.

In [ ]:
initial_request["num_results"]

In [ ]:
len(initial_request["results"])

In [ ]:
initial_request["results"][0]

Note that this just gives us the url for the web page we want. 

We can use the python `requests` module to actually get the web page.

In [ ]:
res = requests.get('https://biggs.house.gov/media/press-releases/congressman-biggs-reintroduces-ending-common-core-and-expanding-school-choice')

This just gives us the html source of the web page. So there's work to do

In [ ]:
print(res.text)

In [ ]:
from IPython.core.display import display, HTML
display(HTML(res.text))

Changing the offset can give us more results

In [ ]:
offset = 20
second_request = requests.get(propublica_query.format(offset), headers = headers).json()

## Reddit

You can download data from reddit. A page that explains how to do it is [here](https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c): 

In [ ]:
import requests

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('<CLIENT_ID>', '<SECRET_TOKEN>')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': '<USERNAME>',
        'password': '<PASSWORD>'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

In [ ]:
res = requests.get("https://oauth.reddit.com/r/Northwestern",
                   headers=headers)

for post in res.json()["data"]["children"]:
    print(post["data"]["title"])

In [ ]:
post["data"]["selftext"]